In [3]:
import pandas as pd
import numpy as np

In [4]:
pytess = pd.read_pickle('pytesseract_results.dat')
pdfminer = pd.read_pickle('pdfminer_ables.dat')

In [25]:
date = []
LOAN_NUMBER = []
project = []
amount = []

pytess_label = pytess.index.values
for i in range(len(pdfminer)):
    index = []
    for j in range(len(pdfminer.iloc[i,0])):
        if pdfminer.iloc[i,0][j] == '_':
            index.append(j)
        
    date.append(pdfminer.iloc[i,0][:index[2]].replace("_",""))
    
    if i in pytess_label:
        label_read = np.where(pytess_label == i)
        label_int = int(label_read[0])
        document = pytess["reparse"].iloc[label_int]
        
    else:
        document = pdfminer["raw_parse"].iloc[i]

    start = document.find('NUMBER')
    LOAN_NUMBER.append(document[start+7:start+15].replace("\n",""))

    openup = 0
    closedown = 2
    for i in range(len(document)): 
        if document[i] == '(':
            openup = i
            break
    for j in range(len(document)):
        if document[j] == ')':
            closedown = j
            break
    project.append(document[openup+1: closedown])
    
    if document.find('The Bank agrees to') == -1:
        if document.find('The  Bank  agrees  to') == -1:
            amount_keyword = document.find('The  Bank agrees  to')
        else:
            amount_keyword = document.find('The  Bank  agrees  to')
    else:
        amount_keyword = document.find('The Bank agrees to')
    
    amount_open = document[amount_keyword:].find('(')
    amount_close = document[amount_keyword:].find(')')
    amount.append(document[amount_keyword + amount_open+1:amount_keyword + amount_close])
    

In [26]:
country_code = []

for code in LOAN_NUMBER:
    if '-' in code:
        start2 = code.find('-')
        country_code.append(code[start2+1:])
        continue
        
    if ' ' in code:
        start = code.find(' ')
        country_code.append(code[start+1:])
        continue
 
    else:
        country_code.append(code)


In [27]:
for i in range(len(date)):
        date[i] = date[i].replace("january","-1-")
        date[i] = date[i].replace("february","-2-")
        date[i] = date[i].replace("march","-3-")
        date[i] = date[i].replace("april","-4-")
        date[i] = date[i].replace("may","-5-")
        date[i] = date[i].replace("june","-6-")
        date[i] = date[i].replace("july","-7-")
        date[i] = date[i].replace("august","-8-")
        date[i] = date[i].replace("september","-9-")
        date[i] = date[i].replace("october","-10-")
        date[i] = date[i].replace("november","-11-")
        date[i] = date[i].replace("december","-12-")

In [28]:
dictionary = {'date': date,'country_code':country_code,'loan_number':LOAN_NUMBER,'amount':amount,'project':project}
df = pd.DataFrame(data = dictionary)
df

,date,country_code,loan_number,amount,project
0,1990-4-24,IVC,3186 IVC,"$80,000,000",Forestry Sector Project
1,1990-4-24,SLU,3184 SLU,"$2,500,000",Water Supply Project
2,1990-4-25,POL,3190 POL,"$18,000,000",Environment Management Project
3,1990-4-30,IND,3180 IND,"$329,000,000",Rural Electrification Project
4,1990-4-30,IND,3182 IND,"$350,000,000",Third Telecommunications Project
...,...,...,...,...,...
3200,2019-9-13,MN,9003-MN,"$20,000,000",Economic Management Support operation Second...
3201,2019-9-13,CO,8798-CO,"$41,900,000",Enhancing Waterway Connectivity and Water Serv...
3202,2019-9-23,RO,9005-RO,"500,000,000 Euro",Romania Health Program for Results
3203,2019-9-25,SR,8985-SR,"823,000,000",Competitiveness and Sector Diversification Pro...


In [29]:
df.to_csv('dblap.csv')

In [24]:
df["amount"].iloc[18]

'$80,000,000'